In [1]:
from square.client import Client

In [2]:
## Add this to the first block in your note book to display json
# another option: 
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

# Since this is copy-pasted wrongly(mostly) at a lot of places across the web, i'm putting the fixed, updated version here, mainly for self-reference


## To use this function, call this, this now works even when you have a print statement before or after the RenderJSON call
# RenderJSON(your_json)
# RenderJSON(json.loads(result.text))

In [4]:
# from square.client import Client
sandbox_token = "EAAAECCddSeDNsweJXdjNVux8FTfVgMUXAroCh7VcOuoPlgjYb7MrkULH8DZrZmJ"

client_sandbox = Client(
    access_token = sandbox_token,
    environment='sandbox',
)

In [3]:
production_token = "EAAAEfz2MN9eAPDmBIuiC4ZT2ulokQjU3JIKHtBxm-R4TCAZgh4QWM2Svb0pRQNH"

client_production = Client(
    access_token = production_token,
    environment='production',
)

In [7]:
location_id = "LCN0KMDE4Y5WR"
customer_id = "RAWRMY0ZRWZVKC51JH2JQT2PKW"

line_items_list = [
        {
          "quantity": "3",
          "catalog_object_id": "IRC3NJWX5QBVIW7U736JPRCC"
        },
          {
          "quantity": "5",
          "catalog_object_id": "KYZZN57QA5JHS6CWZBMWMEE6"
        }
      ]

result = client_sandbox.checkout.create_checkout(
  location_id = location_id,
  body = {
    "idempotency_key": str(uuid.uuid1()),
    "order": {
      "order": {
        "location_id": location_id,
        "customer_id": customer_id,
        "line_items": line_items_list        
      }
    }
  }
)

if result.is_success():
  RenderJSON(result.body)
elif result.is_error():
  print(result.errors)

In [8]:
result = client_sandbox.orders.retrieve_order("mZQcvAkRHb2bDQPHp1jmBY96cg4F")
RenderJSON(result.body)

In [11]:
# pay the order
order_id ="mZQcvAkRHb2bDQPHp1jmBY96cg4F"
amount = 15500

result_create_payment = client_sandbox.payments.create_payment(
      body = {
        "source_id": "cnon:card-nonce-ok",
        "idempotency_key": str(uuid.uuid4()),
        "amount_money": {
          "amount": amount,
          "currency": "USD"
        },
        "order_id": order_id,
#           "note": note_payment
      }
    )
RenderJSON(result_create_payment.body)

In [12]:
result = client_sandbox.payments.get_payment("dIjp7s8hSKlhEtN80vQeL5hJCU9YY")
RenderJSON(result.body)

In [14]:
# pay order
result = client_sandbox.orders.pay_order(
  order_id = "mZQcvAkRHb2bDQPHp1jmBY96cg4F",
  body = {
    "idempotency_key": str(uuid.uuid1()),
      "payment_ids": [
      "dIjp7s8hSKlhEtN80vQeL5hJCU9YY"
    ]
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

result
# [{'category': 'INVALID_REQUEST_ERROR', 'code': 'BAD_REQUEST', 'detail': 'The order is already paid.'}]


[{'category': 'INVALID_REQUEST_ERROR', 'code': 'BAD_REQUEST', 'detail': 'The order is already paid.'}]


In [17]:
result = client_sandbox.refunds.refund_payment(
body = {
    "idempotency_key": str(uuid.uuid4()),
    "amount_money": {
      "amount": 1,
      "currency": "USD"
    },
    "payment_id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
    "reason": "REFUND TEST"
  }
)

if result.is_success():
  RenderJSON(result.body)
elif result.is_error():
  print(result.errors)


# result
# ⊖{
#     "refund": ⊖{
#         "amount_money": ⊖{
#             "amount": 1,
#             "currency": "USD"
#         },
#         "created_at": "2021-05-28T19:50:49.651Z",
#         "id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY_TA24APiTljapc7SPhIdnfqf8XhB6dfXWGV7nSsNMwlb",
#         "location_id": "LCN0KMDE4Y5WR",
#         "order_id": "0KvZLkJy6QWp6QGpKTrEg3CIth4F",
#         "payment_id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
#         "reason": "REFUND TEST",
#         "status": "PENDING",
#         "updated_at": "2021-05-28T19:50:49.651Z"
#     }
# }

In [19]:
# complete now
result = client_sandbox.refunds.get_payment_refund("dIjp7s8hSKlhEtN80vQeL5hJCU9YY_TA24APiTljapc7SPhIdnfqf8XhB6dfXWGV7nSsNMwlb")

if result.is_success():
  RenderJSON(result.body)
elif result.is_error():
  print(result.errors)

In [20]:
# multiple times ask for refund
# it works
result = client_sandbox.refunds.refund_payment(
body = {
    "idempotency_key": str(uuid.uuid4()),
    "amount_money": {
      "amount": 1,
      "currency": "USD"
    },
    "payment_id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
    "reason": "REFUND TEST"
  }
)

if result.is_success():
  RenderJSON(result.body)
elif result.is_error():
  print(result.errors)


# result
# ⊖{
#     "refund": ⊖{
#         "amount_money": ⊖{
#             "amount": 1,
#             "currency": "USD"
#         },
#         "created_at": "2021-05-28T19:53:07.816Z",
#         "id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY_gL45RFnmwjhBM0hPTsVhYaj2DfMUBsAxHLew16y2ilO",
#         "location_id": "LCN0KMDE4Y5WR",
#         "order_id": "Q4vvqaplA4lRsyu23OSSapRAHc4F",
#         "payment_id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
#         "reason": "REFUND TEST",
#         "status": "PENDING",
#         "updated_at": "2021-05-28T19:53:07.816Z"
#     }
# }

In [30]:
# now the payment record has several refund history
result = client_sandbox.payments.get_payment("dIjp7s8hSKlhEtN80vQeL5hJCU9YY")
RenderJSON(result.body)


# result
# ⊖{
#     "payment": ⊖{
#         "amount_money": ⊕{2 items},
#         "approved_money": ⊕{2 items},
#         "card_details": ⊕{7 items},
#         "created_at": "2021-05-28T18:59:58.854Z",
#         "delay_action": "CANCEL",
#         "delay_duration": "PT168H",
#         "delayed_until": "2021-06-04T18:59:58.854Z",
#         "id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
#         "location_id": "LCN0KMDE4Y5WR",
#         "order_id": "mZQcvAkRHb2bDQPHp1jmBY96cg4F",
#         "processing_fee": ⊕[1 item],
#         "receipt_number": "dIjp",
#         "receipt_url": "https://squareupsandbox.com/receipt/preview/dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
#         "refund_ids": ⊖[
#             "dIjp7s8hSKlhEtN80vQeL5hJCU9YY_TA24APiTljapc7SPhIdnfqf8XhB6dfXWGV7nSsNMwlb",
#             "dIjp7s8hSKlhEtN80vQeL5hJCU9YY_gL45RFnmwjhBM0hPTsVhYaj2DfMUBsAxHLew16y2ilO"
#         ],
#         "refunded_money": ⊕{2 items},
#         "risk_evaluation": ⊕{2 items},
#         "source_type": "CARD",
#         "status": "COMPLETED",
#         "total_money": ⊕{2 items},
#         "updated_at": "2021-05-28T19:53:07.816Z",
#         "version_token": "qH81L5POgKtL33C4OH2S8JnlqbVJ5hZWdiJIMwJoFxk6o"
#     }
# }

In [31]:
# try something bold (failed)

# first create a refund
result_refund = client_sandbox.refunds.refund_payment(
body = {
    "idempotency_key": str(uuid.uuid4()),
    "amount_money": {
      "amount": 10,
      "currency": "USD"
    },
    "payment_id": "dIjp7s8hSKlhEtN80vQeL5hJCU9YY",
    "reason": "REFUND TEST UPDATE ORDER"
  }
)
RenderJSON(result_refund.body)

# try to "immediately update the ad hoc order"
order_id = result_refund.body.get("refund").get("order_id")
result_adhoc_order = client_sandbox.orders.retrieve_order(order_id)
version = result_adhoc_order.body.get("order").get("version")
result_order_update = client_sandbox.orders.update_order(
    order_id=order_id,
    body={
    "order": {
      "location_id": location_id,
      "reference_id": "UPDATE TEST",
        "version": version
    }
    }
)
                                                 
RenderJSON(result_order_update.body)


# result
# {
#     "refund": ⊕{9 items}
# }

# ⊖{
#     "errors": ⊖[
#        ⊖{
#             "category": "INVALID_REQUEST_ERROR",
#             "code": "BAD_REQUEST",
#             "detail": "Order with id `QqSS1OwLESUetzKpVnaI2nd6Eb4F` has status `COMPLETED` and cannot be updated."
#         }
#     ]
# }